In [1]:
import requests
import pandas as pd
import datetime
import json
import os
import db_dtypes
import re
import time
from dotenv import load_dotenv
from dotenv import load_dotenv
from typing import Optional
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from logging_export import logger

# Define Functions

In [2]:
def upload_to_bucket(storage_client, 
                     bucket_name,
                     df_list):

    
    my_bucket = storage_client.bucket(bucket_name)
    
    # Check if the specified bucket exists in the project or not
    if not my_bucket.exists():
        # If the bucket exists
        try:
            logger.info(f'Bucket - {bucket_name} is not found.')
            logger.info(f'Creating new bucket: {bucket_name}')
            my_bucket.create()
        except Exception as e:
            logger.error(f'Error creating bucket: {e}')

        logger.info(f'Success creating bucket: {bucket_name}')

    else:
        logger.info(f'Bucket - {bucket_name} is found.')
        
    if df_list is None:
            logger.warning('Please key in value for item.')
            
    else:
        try:
            # To create a list to store the gsutil uri 
            gsutil_uri_list = []
            for item in df_list:
                # timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S ')
                # blob_name = item.name + " " + timestamp
                blob_name = item.name
                blob = my_bucket.blob(blob_name)
                blob.upload_from_string(item.to_csv(index=False), 'text/csv')
                logger.info(f'SUCCESS: {blob} has successfully uploaded to {my_bucket}.')

                gsutil_uri = f"gs://{bucket_name}/{blob_name}"
                gsutil_uri_list.append(gsutil_uri)
            return gsutil_uri_list
                
        except Exception as e:
            return logger.error(f'Error uploading file to {bucket_name}: {e}')

In [3]:
def upload_to_bigquery(client, dataset_name, table_name, job_config, gsutil_uri):
    
    try:
        for i in range(len(table_name)): 
            # Look up for dataset
            dataset_ref = client.dataset(dataset_name)
            try:
                targeted_dataset = client.get_dataset(dataset_ref)
                logger.info(f'Dataset {targeted_dataset.dataset_id} already exists.')
            # Create new dataset if not found
            except NotFound:
                targeted_dataset = client.create_dataset(dataset_ref)
                logger.info(f'Dataset {targeted_dataset.dataset_id} created.')

            # Look up for table
            table_ref = dataset_ref.table(table_name[i])
            try:
                targeted_table = client.get_table(table_ref)
                logger.info(f'Table {targeted_table.table_id} already exists.')
            # Create new table if not found
            except NotFound:
                targeted_table = client.create_table(table_ref)
                logger.info(f'Table {targeted_table.table_id} created.')

            # Upload the data to bigquery table using gsutil URI
            load_job = client.load_table_from_uri(gsutil_uri[i],
                                                  targeted_table, 
                                                  job_config=job_config)

            logger.info(load_job.result())
        return logger.info("Done")
        
    except Exception as e:
        return logger.error(f"Fail to upload :{e}")

# Execution Parts
## Load Environment Variables

In [4]:
load_dotenv()

# Convert the credentials to .json file for the usage of GOOGLE_APPLICATION_CREDENTIALS
CREDENTIALS = json.loads(os.getenv('CREDENTIALS'))

# Check if there's an existing credentials file
with open('credentials.json', 'w') as cred_file:
    json.dump(CREDENTIALS, cred_file)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ='credentials.json'

storage_client = storage.Client()
bucket_name = 'project-airbnb'

## Read data from .csv file

## Read data from .csv file
### Tokyo Listings

In [14]:
# Read the .csv file
filepath = r"./tokyo_listings.csv"
df_tokyo = pd.read_csv(filepath,sep=',')
df_tokyo = df_tokyo.dropna(axis=1,how='all').drop_duplicates().dropna()

columns_to_drop = ['scrape_id', 'name', 'host_about', 'neighborhood_overview', 'amenities']
df_tokyo = df_tokyo.drop(columns_to_drop, axis=1)
    
df_tokyo = df_tokyo.reset_index(drop=True)
df_tokyo.name = 'tokyo-listings'

In [15]:
df_list = [] 
df_list.append(df_tokyo)

## Load Data to Google Bucket

In [16]:
gsutil_uri_list = upload_to_bucket(storage_client=storage_client,
                                   bucket_name=bucket_name,
                                   df_list=df_list)

INFO - Bucket - project-airbnb is found.
INFO - SUCCESS: <Blob: project-airbnb, tokyo-listings, 1705675768921649> has successfully uploaded to <Bucket: project-airbnb>.


## Load Data to Google BigQuery

In [17]:
bq_client = bigquery.Client()
dataset_name = "airbnb"
table_name = ["tokyo-listing"]
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.CSV,
                                    write_disposition='WRITE_TRUNCATE',
                                    skip_leading_rows=1,
                                    autodetect=True,)
gsutil_uri = gsutil_uri_list

In [18]:
upload_to_bigquery(client=bq_client,
                   dataset_name=dataset_name,
                   table_name=table_name,
                   job_config=job_config,
                   gsutil_uri=gsutil_uri)

INFO - Dataset airbnb already exists.
INFO - Table tokyo-listing already exists.
INFO - LoadJob<project=personal-project-401309, location=US, id=6bd5a2c2-17e3-419a-b212-45a15fd83158>
INFO - Done


## Extracting insights from the data via data query
### Tokyo Listings

In [7]:
# Construct a BigQuery client object.
client = bigquery.Client()

sql = f"""
SELECT 
  neighbourhood_cleansed,
  AVG(availability_30) AS avg_avail_30,
  AVG(availability_60) AS avg_avail_60,
  AVG(availability_90) AS avg_avail_90,
  AVG(availability_365) AS avg_avail_365
FROM `personal-project-401309.airbnb.tokyo-listing`
GROUP BY neighbourhood_cleansed
ORDER BY avg_avail_30 ASC;
"""
availability_trends = client.query(sql).to_dataframe()  # Make an API request.
availability_trends 

,neighbourhood_cleansed,avg_avail_30,avg_avail_60,avg_avail_90,avg_avail_365
0,Shibuya Ku,6.994709,17.121693,30.052910,124.412698
1,Meguro Ku,7.000000,21.285714,39.380952,128.000000
2,Kita Ku,7.762712,19.288136,33.508475,114.440678
3,Shinjuku Ku,8.439614,20.589372,34.997585,121.671498
4,Katsushika Ku,8.907692,21.307692,34.076923,129.938462
5,Taito Ku,9.042553,21.978723,38.112462,150.750760
6,Nerima Ku,9.181818,22.500000,36.727273,132.090909
7,Minato Ku,9.296875,22.390625,34.945313,89.375000
8,Toshima Ku,9.392157,21.852941,35.931373,143.372549
9,Bunkyo Ku,9.593750,25.781250,43.468750,235.890625


In [12]:
sql = f"""
SELECT 
  neighbourhood_cleansed,
  AVG(price) AS average_price,
  MIN(price) AS min_price,
  MAX(price) AS max_price,
  AVG(review_scores_rating) AS average_rating,
  COUNT(*) AS number_of_listings
FROM `personal-project-401309.airbnb.tokyo-listing`
GROUP BY neighbourhood_cleansed
ORDER BY average_rating DESC, average_price ASC;
"""
pricing_strategy = client.query(sql).to_dataframe()  # Make an API request.
pricing_strategy 

,neighbourhood_cleansed,average_price,min_price,max_price,average_rating,number_of_listings
0,Suginami Ku,17869.848101,4614,45000,4.764557,79
1,Ota Ku,30524.371795,4114,131429,4.764359,78
2,Katsushika Ku,35307.923077,2543,116000,4.756000,65
3,Arakawa Ku,26218.636364,2900,200000,4.750000,33
4,Taito Ku,32098.440729,3500,201829,4.743374,329
5,Nakano Ku,30954.587629,4500,292857,4.742784,97
6,Shinjuku Ku,37679.212560,5000,225714,4.741787,414
7,Chofu Shi,28286.000000,28286,28286,4.740000,1
8,Setagaya Ku,21260.032680,3500,199120,4.739804,153
9,Meguro Ku,23522.476190,5371,94757,4.734286,21


In [13]:
sql = f"""
SELECT 
  neighbourhood_cleansed,
  ROUND(AVG(review_scores_rating), 2) AS average_rating,
  ROUND(AVG(review_scores_cleanliness), 2) AS avg_cleanliness,
  ROUND(AVG(review_scores_communication), 2) AS avg_communication
FROM `personal-project-401309.airbnb.tokyo-listing`
GROUP BY neighbourhood_cleansed
ORDER BY average_rating DESC, average_rating ASC;
"""
competitive_analysis = client.query(sql).to_dataframe()  # Make an API request.
competitive_analysis 

,neighbourhood_cleansed,average_rating,avg_cleanliness,avg_communication
0,Katsushika Ku,4.76,4.66,4.89
1,Suginami Ku,4.76,4.75,4.85
2,Ota Ku,4.76,4.76,4.84
3,Arakawa Ku,4.75,4.73,4.89
4,Setagaya Ku,4.74,4.69,4.85
5,Shinjuku Ku,4.74,4.71,4.87
6,Taito Ku,4.74,4.70,4.88
7,Nakano Ku,4.74,4.71,4.84
8,Chofu Shi,4.74,4.56,4.85
9,Meguro Ku,4.73,4.73,4.90
